## 🤖 Criando um agente conversacional

Instalando as bibliotecas de LangChain:

In [ ]:
%pip install langchain
%pip install langchain-groq

Criando ambiente para armazenar a chave de API do Groq para uso dos modelos gratuítos:

In [ ]:
import os

api_key = 'YOUR_GROQ_API_KEY'
os.environ['GROQ_API_KEY'] = api_key

Configurando resposta inicial do chatbot:

In [ ]:
from langchain_groq import ChatGroq

chat = ChatGroq(model='llama-3.3-70b-specdec')
response = chat.invoke('Hello, how are you?')

print(response.content)